# 5 tips for working with time series in Python
> At some point in his/her career, any Data Scientist has to be able to manipulate time series data. I have been working as a Data Scientist and Quant Researcher for the last 14 months and I found little “cooking tips” for working with this type of data. Today, I would like to share some of those tips.

- toc:true
- badges: true
- comments: true
- author: Alejandro PS
- categories: [python, numpy, pandas]

*This is a repost from my medium article [5 tips for working with time series in Python](https://alejandrops.medium.com/5-tips-for-working-with-time-series-in-python-d889109e676d)*

---

## Required knowledge
This post is pretty easy to follow if you already have some basic knowledge of Pandas, NumPy and Python. I will not go into much details with the theoretical stuff but use the resources at the end or ask a question if you need clarification about some particular concept.


## Removing noise with the Fourier Transform
It is often the case that we need to study the underlying process that drives a particular time series. To do that, we may want to remove the noise of the time series and analyze the signal.

The Fourier Transform can help us achieve this objective. By moving our time series from the time domain to the frequency domain, we can filter out the frequencies that pollute the data. Then, we just have to apply the inverse Fourier transform to get a filtered version of our time series.

*Note: the code presented in this section is a slightly modified version of Steven L. Brunton code. See References section to find the original code and explanation.*

### 2.1. The code
The following gist contains the necessary code to remove the noise using the Fourier Transform:

In [1]:
def fft_denoiser(x, n_components, to_real=True):
    """Fast fourier transform denoiser.
    
    Denoises data using the fast fourier transform.
    
    Parameters
    ----------
    x : numpy.array
        The data to denoise.
    n_components : int
        The value above which the coefficients will be kept.
    to_real : bool, optional, default: True
        Whether to remove the complex part (True) or not (False)
        
    Returns
    -------
    clean_data : numpy.array
        The denoised data.
        
    References
    ----------
    .. [1] Steve Brunton - Denoising Data with FFT[Python]
       https://www.youtube.com/watch?v=s2K1JfNR7Sc&ab_channel=SteveBrunton
    
    """
    n = len(x)
    
    # compute the fft
    fft = np.fft.fft(x, n)
    
    # compute power spectrum density
    # squared magnitud of each fft coefficient
    PSD = fft * np.conj(fft) / n
    
    # keep high frequencies
    _mask = PSD > n_components
    fft = _mask * fft
    
    # inverse fourier transform
    clean_data = np.fft.ifft(fft)
    
    if to_real:
        clean_data = clean_data.real
    
    return clean_data

### Usage example
![](img/5tips_ts/fourier.png)

## Removing noise with the Kalman Filter
With the Fourier Transform we obtain the frequencies that exist in a given time series, but we do not have any information of when these frequencies occur in time. This means that, in its basic form, the Fourier Transform is not the best choice for non-stationary time series.

For example, financial time series are considered non-stationary (although any attempt [to prove it statistically is doomed](https://towardsdatascience.com/non-stationarity-and-memory-in-financial-markets-fcef1fe76053)), thus making Fourier a bad choice.

At this point, we can choose to apply the Fourier Transform in a rolling-basis or to go with a [Wavelet Transform](http://users.rowan.edu/~polikar/WTpart1.html). But there is a much more interesting algorithm **called Kalman Filter**.

The [Kalman Filter](https://www.cs.unc.edu/~welch/media/pdf/kalman_intro.pdf) is essentially a [Bayesian Linear Regression](https://users.aalto.fi/~ssarkka/pub/cup_book_online_20131111.pdf) that can optimally estimate the hidden state of a process using its observable variables.

By carefully selecting the right parameters, one can tweak the algorithm to extract the underlying signal.

### 3.1. The code
[I created a small library](https://github.com/Xylambda/kalmanfilter) that contains a univariate Kalman Filter that can be used to extract the signal. In the README you will find the particular set of parameters I used. You can also use [PyKalman](https://pykalman.github.io/).

### 3.2. Example
![](img/5tips_ts/kalman.png)

## Dealing with Outliers
Outliers are usually undesirable because they deeply affect our conclusions if we are not careful when dealing with them. For example, the Pearson correlation formula can have a very different result if there are large enough outliers in our data.

Outlier analysis and filtering in time series requires a more sophisticated approach than in normal data, since **you cannot use future information to filter past outliers.**

One quick way to remove outliers is doing it in a rolling/expanding basis. A common algorithm to find outliers is computing the mean and standard deviation of our data and check which values are n standard deviations above or below the mean (typically, n is set to 3). Those values are then marked as outliers.

### The code
The following code allows you to filter outliers using the aforementioned algorithm but in rolling or expanding mode to avoid [look-ahead bias](https://corporatefinanceinstitute.com/resources/knowledge/finance/look-ahead-bias/).

In [2]:
def basic_filter(data, mode='rolling', window=262, threshold=3):
    """Basic Filter.
    
    Mark as outliers the points that are out of the interval:
    (mean - threshold * std, mean + threshold * std ).
    
    Parameters
    ----------
    data : pandas.Series
        The time series to filter.
    mode : str, optional, default: 'rolling'
        Whether to filter in rolling or expanding basis.
    window : int, optional, default: 262
        The number of periods to compute the mean and standard
        deviation.
    threshold : int, optional, default: 3
        The number of standard deviations above the mean.
        
    Returns
    -------
    series : pandas.DataFrame
        Original series and marked outliers.
    """
    msg = f"Type must be of pandas.Series but {type(data)} was passed."
    assert isinstance(data, pd.Series), msg
    
    series = data.copy()
    
    # rolling/expanding objects
    pd_object = getattr(series, mode)(window=window)
    mean = pd_object.mean()
    std = pd_object.std()
    
    upper_bound = mean + threshold * std
    lower_bound = mean - threshold * std
    
    outliers = ~series.between(lower_bound, upper_bound)
    # fill false positives with 0
    outliers.iloc[:window] = np.zeros(shape=window)
    
    series = series.to_frame()
    series['outliers'] = np.array(outliers.astype('int').values)
    series.columns = ['Close', 'Outliers']
    
    return series

### Example
Playing with the parameters you can fine-tune your analysis. Here is an example using the default values of the function.

![](img/5tips_ts/outliers.png)

## The right way to normalize time series data.
Many posts use the classical fit-transform approach with time series as if they could be treated as normal data. As with outliers, you cannot use future information to normalize data from the past unless you are 100% sure the values you are using to normalize are constant over time.

The right way to normalize time series is in a rolling/expanding basis.

### The code
I used Sklearn API to create a class that allows you to normalize data avoiding look-ahead bias. Because it inherits BaseEstimator and TransformerMixin it is possible to embed this class in a Sklearn pipeline.

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin


class RollingStandardScaler(BaseEstimator, TransformerMixin):
    """Rolling standard Scaler
    
    Standardized the given data series using the mean and std 
    commputed in rolling or expanding mode.
    
    Parameters
    ----------
    window : int
        Number of periods to compute the mean and std.
    mode : str, optional, default: 'rolling'
        Mode 
        
    Attributes
    ----------
    pd_object : pandas.Rolling
        Pandas window object.
    w_mean : pandas.Series
        Series of mean values.
    w_std : pandas.Series
        Series of std. values.
    """
    def __init__(self, window, mode='rolling'):
        self.window = window
        self.mode = mode
        
        # to fill in code
        self.pd_object = None
        self.w_mean = None
        self.w_std = None
        self.__fitted__ = False
        
    def __repr__(self):
        return f"RollingStandardScaler(window={self.window}, mode={self.mode})"
        
    def fit(self, X, y=None):
        """Fits.
        
        Computes the mean and std to be used for later scaling.
        
        Parameters
        ----------
        X : array-like of shape (n_shape, n_features)
            The data used to compute the per-feature mean and std. Used for
            later scaling along the feature axis.
        y
            Ignored.
        """
        self.pd_object = getattr(X, self.mode)(self.window)
        self.w_mean = self.pd_object.mean()
        self.w_std = self.pd_object.std()
        self.__fitted__ = True
        
        return self
    
    def transform(self, X):
        """Transforms.
        
        Scale features of X according to the window mean and standard 
        deviation.
        
        Paramaters
        ----------
        X : array-like of shape (n_shape, n_features)
            Input data that will be transformed.
        
        Returns
        -------
        standardized : array-like of shape (n_shape, n_features)
            Transformed data.
        """
        self._check_fitted()
        
        standardized = X.copy()
        return (standardized - self.w_mean) / self.w_std
    
    def inverse_transform(self, X):
        """Inverse transform
        
        Undo the transform operation
        
        Paramaters
        ----------
        X : array-like of shape (n_shape, n_features)
            Input data that will be transformed.
        
        Returns
        -------
        standardized : array-like of shape (n_shape, n_features)
            Transformed (original) data.
        """
        self._check_fitted()
        
        unstandardized = X.copy()
        return  (unstandardized * self.w_std) + self.w_mean
        
    def _check_fitted(self):
        """ Checks if the algorithm is fitted. """
        if not self.__fitted__:
            raise ValueError("Please, fit the algorithm first.")

### Example
![](img/5tips_ts/normalization.png)

## A flexible way to compute returns.
The last tip is focused on quantitative analysis of financial time series. Working with returns is the first thing you learn as a quant researcher. Hence, it is necessary to have a basic framework to quickly compute log and arithmetic returns in different periods of time.

Also, when filtering financial time series, the ideal procedure filters returns first and then goes back to prices. So you are free to add this step to the code from section 4.

### The code
The following gist contains a basic framework to compute returns

In [4]:
""" Utils functions. """

import numpy as np
import pandas as pd


def compute_returns(data, periods=1, log=False, relative=True):
    """Computes returns.

    Calculates the returns of a given dataframe for the given period. The 
    returns can be computed as log returns or as arithmetic returns
    
    Parameters
    ----------
    data : pandas.DataFrame or pandas.Series
        The data to calculate returns of.
    periods : int
        The period difference to compute returns.
    log : bool, optional, default: False
        Whether to compute log returns (True) or not (False).
    relative : bool, optional, default: True
        Whether to compute relative returns (True) or not 
        (False).
    
    Returns
    -------
    ret : pandas.DataFrame or pandas.Series
        The computed returns.
    
    """
    if log:
        if not relative:
            raise ValueError("Log returns are relative by definition.")
        else:
            ret = _log_returns(data, periods)
    else:
        ret = _arithmetic_returns(data, periods, relative)

    return ret


def _arithmetic_returns(data, periods, relative):
    """Arithmetic returns."""
    # to avoid computing it twice
    shifted = data.shift(periods)
    ret = (data - shifted) 
    
    if relative:
        return ret / shifted
    else:
        return ret


def _log_returns(data, periods):
    """Log returns."""
    return np.log(data / data.shift(periods))

### Example
![](img/5tips_ts/returns.png)

These are some of the tips I find more useful in my day-to-day basis. I really hope you find something interesting in this post and, if you find any error or would like to discuss any concept, please leave a comment and I will answer as soon as possible.

## References
1. Callum Ballard — [Making Matplotlib Beautiful By Default](https://towardsdatascience.com/making-matplotlib-beautiful-by-default-d0d41e3534fd).
2. Steven L. Brunton — [Denoising Data with FFT [Python]](https://www.youtube.com/watch?v=s2K1JfNR7Sc&ab_channel=SteveBrunton).
3. Greg Welch, Gary Bishop — [An introduction to the Kalman Filter](https://www.cs.unc.edu/~welch/media/pdf/kalman_intro.pdf).
4. Simo Särkkä — Bayesian filtering and smoothing.
5. Yves-Laurent Kom Samo — [Stationarity and Memory in Financial Markets](https://towardsdatascience.com/non-stationarity-and-memory-in-financial-markets-fcef1fe76053).
6. Robi Polikar — [The Wavelet Tutorial](http://users.rowan.edu/~polikar/WTtutorial.html).